In [24]:
import mdptoolbox
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as ss
import warnings
warnings.filterwarnings('ignore', category=ss.SparseEfficiencyWarning)

In [26]:
results = []
for alpha in np.linspace(0.3, 0.45, 4):
    result = []
    maxForkLen = 75
    numOfStates = (maxForkLen+1) * (maxForkLen+1) * 3
    print('alpha: ', alpha)
    result.append(alpha)
    alphaPower = alpha
    gammaRatio = 0
    irrelevant = 0; relevant = 1; active = 2;
    choices = 4
    adopt = 0; override = 1; match = 2; wait = 3;
    P = []; Rs = []; Rh = [];
    for _ in range(choices):
        P.append(ss.csr_matrix(np.zeros(shape=(numOfStates, numOfStates))))
        Rs.append(ss.csr_matrix(np.zeros(shape=(numOfStates, numOfStates))))
        Rh.append(ss.csr_matrix(np.zeros(shape=(numOfStates, numOfStates))))
    # generate a state to integer mapping and list of states
    state_mapping = {}
    states = []
    count = 0
    for a in range(maxForkLen+1):
        for h in range(maxForkLen+1):
            for fork in range(3):
                state_mapping[(a, h, fork)] = count
                states.append((a, h, fork))
                count += 1
    # adopt
    P[adopt][:, state_mapping[1, 0, irrelevant]] = alphaPower
    P[adopt][:, state_mapping[0, 1, irrelevant]] = 1 - alphaPower
    for state_index in range(numOfStates):
        if state_index % 2000 == 0:
            print('processing state', state_index)
        a, h, fork = states[state_index]

        # adopt rewards
        Rh[adopt][state_index, state_mapping[1, 0, irrelevant]] = h
        Rh[adopt][state_index, state_mapping[0, 1, irrelevant]] = h

        # override
        if a > h:
            P[override][state_index, state_mapping[a-h, 0, irrelevant]] = alphaPower
            Rs[override][state_index, state_mapping[a-h, 0, irrelevant]] = h+1
            P[override][state_index, state_mapping[a-h-1, 1, relevant]] = 1 - alphaPower
            Rs[override][state_index, state_mapping[a-h-1, 1, relevant]] = h+1
        else:
            P[override][state_index, 0] = 1
            Rh[override][state_index, 0] = 10000

        # wait
        if (fork != active) and (a < maxForkLen) and (h < maxForkLen):
            P[wait][state_index, state_mapping[a+1, h, irrelevant]] = alphaPower
            P[wait][state_index, state_mapping[a, h+1, relevant]] = 1 - alphaPower
        elif (fork == active) and (a > h) and (h > 0) and (a < maxForkLen) and (h < maxForkLen): 
            P[wait][state_index, state_mapping[a+1, h, active]] = alphaPower
            P[wait][state_index, state_mapping[a-h, 1, relevant]] = gammaRatio*(1-alphaPower)
            Rs[wait][state_index, state_mapping[a-h, 1, relevant]] = h
            P[wait][state_index, state_mapping[a, h+1, relevant]] = (1-gammaRatio)*(1-alphaPower)
        else:
            P[wait][state_index, 0] = 1
            Rh[wait][state_index, 0] = 10000

        # match
        if (fork == relevant) and (a >= h) and (h > 0) and (a < maxForkLen) and (h < maxForkLen):
            P[match][state_index, state_mapping[a+1, h, active]] = alphaPower
            P[match][state_index, state_mapping[a-h, 1, relevant]] = gammaRatio*(1-alphaPower)
            Rs[match][state_index, state_mapping[a-h, 1, relevant]] = h
            P[match][state_index, state_mapping[a, h+1, relevant]] = (1-gammaRatio)*(1-alphaPower)
        else:
            P[match][state_index, 0] = 1
            Rh[match][state_index, 0] = 10000
    epsilon = 0.0001
    lowRho = 0
    highRho = 1
    while(highRho - lowRho > epsilon/8):
        rho = (highRho + lowRho) / 2;
        Wrho = []
        for i in range(choices):
            Wrho.append((1-rho)*Rs[i] - rho*Rh[i])
        rvi = mdptoolbox.mdp.RelativeValueIteration(P, Wrho, epsilon/8)
        rvi.run()
        lowerBoundPolicy = rvi.policy
        reward = rvi.average_reward
        if reward > 0:
            lowRho = rho
        else:
            highRho = rho
    print('alpha: ', alphaPower, 'lower bound reward:', rho)
    result.append(rho)
    lowerBoundRho = rho
    lowRho = rho
    highRho = min(rho+0.1, 1)
    while (highRho - lowRho) > (epsilon / 8):
        rho = (highRho + lowRho) / 2
        for state_index in range(numOfStates):
            a, h, fork = states[state_index]
            if a == maxForkLen:
                expr = (1-rho)*alphaPower*(1-alphaPower)/(1-2*alphaPower)**2+0.5*((a-h)/(1-2*alphaPower)+a+h)
                Rs[adopt][state_index, state_mapping[1, 0, irrelevant]] = expr
                Rs[adopt][state_index, state_mapping[0, 1, irrelevant]] = expr
                Rs[adopt][state_index, state_mapping[1, 0, irrelevant]] = 0
                Rs[adopt][state_index, state_mapping[0, 1, irrelevant]] = 0
            elif h == maxForkLen:
                expr1 = (1 - np.power(alphaPower/(1-alphaPower), h - a)) * (-1*rho*h)
                expr2 = np.power(alphaPower/(1-alphaPower), h - a) * (1 - rho)
                expr3 = (alphaPower * (1-alphaPower)) / (np.power(1-2*alphaPower, 2)) + (h - a) / (1- 2 * alphaPower)
                expr_total = expr1 + expr2 * expr3
                Rs[adopt][state_index, state_mapping[1, 0, irrelevant]] = expr_total
                Rs[adopt][state_index, state_mapping[0, 1, irrelevant]] = expr_total
                Rh[adopt][state_index, state_mapping[1, 0, irrelevant]] = 0
                Rh[adopt][state_index, state_mapping[0, 1, irrelevant]] = 0
        Wrho = []
        for i in range(choices):
            Wrho.append((1-rho)*Rs[i] - rho*Rh[i])
        rhoPrime = max(lowRho - epsilon/4, 0)
        rvi = mdptoolbox.mdp.RelativeValueIteration(P, Wrho, epsilon/8)
        rvi.run()
        reward = rvi.average_reward
        policy = rvi.policy
        if reward > 0:
            lowRho = rho
        else:
            highRho = rho
    print('alpha: ', alphaPower, 'upper bound reward', rho)
    result.append(rho)
    results.append(result)

alpha:  0.3
processing state 0
processing state 2000
processing state 4000
processing state 6000
processing state 8000
processing state 10000
processing state 12000
processing state 14000
processing state 16000
alpha:  0.3 lower bound reward: 0.29999542236328125
alpha:  0.3 upper bound reward 0.30000762939453124
alpha:  0.35
processing state 0
processing state 2000
processing state 4000
processing state 6000
processing state 8000
processing state 10000
processing state 12000
processing state 14000
processing state 16000
alpha:  0.35 lower bound reward: 0.37076568603515625
alpha:  0.35 upper bound reward 0.37077789306640624
alpha:  0.4
processing state 0
processing state 2000
processing state 4000
processing state 6000
processing state 8000
processing state 10000
processing state 12000
processing state 14000
processing state 16000
alpha:  0.4 lower bound reward: 0.48863983154296875
alpha:  0.4 upper bound reward 0.48865203857421874
alpha:  0.45
processing state 0
processing state 2000
p

In [27]:
results

[[0.3, 0.29999542236328125, 0.30000762939453124],
 [0.35, 0.37076568603515625, 0.37077789306640624],
 [0.4, 0.48863983154296875, 0.48865203857421874],
 [0.45, 0.6681442260742188, 0.6705490112304688]]

In [29]:
i = 0
linspace = [0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25, 0.275, 0.3, 0.325, 0.35, 0.375, 0.4, 0.425, 0.45, 0.475]
for result in results:
    print(linspace[i], ' & {0:.5f} & {1:.5f} \\\\'.format(result[1], result[2]))
    i +=1 

0.275  & 0.30000 & 0.30001 \\
0.3  & 0.37077 & 0.37078 \\
0.325  & 0.48864 & 0.48865 \\
0.35  & 0.66814 & 0.67055 \\
